# 수도권, 비수도권 아파트 매매 수급 동향 크롤링

## 데이터 저장하기 위한 db 접근

### 테이블 생성 (보류)

- 연도, 월, 수도권, 지방권

## 데이터 가져오기 (selenium)

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.alert import Alert
from bs4 import BeautifulSoup
import time

options = webdriver.ChromeOptions()

url = 'https://www.reb.or.kr/r-one/portal/stat/easyStatPage.do'  # 부동산 통계정보 사이트 url
small_driver = webdriver.Chrome(service=Service('/home/yoon/ws/eda/driver/chromedriver'),
                          options=options)

small_driver.get(url)

small_driver.maximize_window()  # 전체화면 크기로 지정

driver = small_driver

# '전국주택가격동향조사' 클릭하기
survey_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="treeStatData"]/ul/li[1]/ul/li[2]/span/a/em')))
survey_btn.click()

# '월간동향' 클릭하기
montly_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="treeStatData"]/ul/li[1]/ul/li[2]/ul/li[1]/span')))
montly_btn.click()

# '아파트' 클릭하기
apart_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="treeStatData"]/ul/li[1]/ul/li[2]/ul/li[1]/ul/li[2]/span')))
apart_btn.click()

# '수급동향' 클릭하기
suppy_demand_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="treeStatData"]/ul/li[1]/ul/li[2]/ul/li[1]/ul/li[2]/ul/li[10]/span/a/em')))
suppy_demand_btn.click()

# '(월) 매매수급동향_아파트' 클릭하기
suppy_demand_montly_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="treeStatData"]/ul/li[1]/ul/li[2]/ul/li[1]/ul/li[2]/ul/li[10]/ul/li[1]/span')))
suppy_demand_montly_btn.click()

time.sleep(4)  # 페이지 로딩 대기

# '분류선택' 에서 모든 체크 해재 클릭하기
check_reset_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="clsBox"]/div[2]/span[2]/button[2]')))
check_reset_btn.click()

# 수도권, 지방권 선택하기
capital_area = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="treeClsData"]/ul/li[2]/span/span[2]')))  # 수도권
capital_area.click()
not_capital_area = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="treeClsData"]/ul/li[3]/span/span[2]')))  # 지방권
not_capital_area.click()

# '검색 기간' 클릭하기
period_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="wrttimeTypeB"]')))
period_btn.click()

# value 값이 2014인 키를 보낸다. (2014년 선택하기)
period_combo_box = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="wrttimeStartYear"]')))
combo_box_list = period_combo_box.find_elements(By.TAG_NAME, 'option')

for idx, year in enumerate(combo_box_list):
    if idx == 2:
        period_combo_box.send_keys(year.get_attribute('value'))

# '조회' 클릭
search_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="addStatTab"]')))
search_btn.click()

# IBSheet 경고창 해제
try:
    alert = WebDriverWait(driver, 10).until(EC.alert_is_present())
    alert.accept()  # 경고 창 닫기
except:
    print("경고 창이 나타나지 않았습니다.")

# beautifulsouop 으로 현재 페이지 html 형태로 가져오기
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')

## 데이터 가져오기 (beautifulsoup)

### 년도, 월 가져오기

In [2]:
date_panel = soup.find('div', class_='IBHeadMid')  # 날짜 판텔
body_panel = date_panel.find_all('tr')  # 'tr' 태그 모두 가져오기
date_column = body_panel[1]  # 1번 째 'tr' 태그에 날짜가 있음.
date_text = date_column.find_all('td')[1:129]  # 문자열로 가져오기

#date_text

for idx, each in enumerate(date_text):
    year = int(each.get_text()[:4])
    if len(each.get_text()) >= 9:
        month = int(each.get_text()[6:8])
    else:
        month = int(each.get_text()[6])
    print(f'연도 : {year}, 월 : {month}')

연도 : 2014, 월 : 1
연도 : 2014, 월 : 2
연도 : 2014, 월 : 3
연도 : 2014, 월 : 4
연도 : 2014, 월 : 5
연도 : 2014, 월 : 6
연도 : 2014, 월 : 7
연도 : 2014, 월 : 8
연도 : 2014, 월 : 9
연도 : 2014, 월 : 10
연도 : 2014, 월 : 11
연도 : 2014, 월 : 12
연도 : 2015, 월 : 1
연도 : 2015, 월 : 2
연도 : 2015, 월 : 3
연도 : 2015, 월 : 4
연도 : 2015, 월 : 5
연도 : 2015, 월 : 6
연도 : 2015, 월 : 7
연도 : 2015, 월 : 8
연도 : 2015, 월 : 9
연도 : 2015, 월 : 10
연도 : 2015, 월 : 11
연도 : 2015, 월 : 12
연도 : 2016, 월 : 1
연도 : 2016, 월 : 2
연도 : 2016, 월 : 3
연도 : 2016, 월 : 4
연도 : 2016, 월 : 5
연도 : 2016, 월 : 6
연도 : 2016, 월 : 7
연도 : 2016, 월 : 8
연도 : 2016, 월 : 9
연도 : 2016, 월 : 10
연도 : 2016, 월 : 11
연도 : 2016, 월 : 12
연도 : 2017, 월 : 1
연도 : 2017, 월 : 2
연도 : 2017, 월 : 3
연도 : 2017, 월 : 4
연도 : 2017, 월 : 5
연도 : 2017, 월 : 6
연도 : 2017, 월 : 7
연도 : 2017, 월 : 8
연도 : 2017, 월 : 9
연도 : 2017, 월 : 10
연도 : 2017, 월 : 11
연도 : 2017, 월 : 12
연도 : 2018, 월 : 1
연도 : 2018, 월 : 2
연도 : 2018, 월 : 3
연도 : 2018, 월 : 4
연도 : 2018, 월 : 5
연도 : 2018, 월 : 6
연도 : 2018, 월 : 7
연도 : 2018, 월 : 8
연도 : 2018, 월 : 9
연도 : 2018, 월 : 10
연

### 수도권, 지방권 아파트 매매 수급 지수 가져오기

- 수도권 = rate_list[0]

- 지방권 = rate_list[1]

In [3]:
# 매매 수급 비율 데이터 가져오기
supply_demand_rate_panel = soup.find_all('div', class_='IBPageOne')[1]  # 1번째 'IBPageOne' 클래스 갸져오기
rate_index = supply_demand_rate_panel.find_all('tr')[1:]  # 수도권 인덱스 = 0, 지방권 인덱스 = 1

for index in rate_index:
    rate_list = index.find_all('td')[1:129]
    for idx, each_rate in enumerate(rate_list):
        first_rate = int(each_rate.get_text().replace('.', ''))  # '.' 없앤 후 정수로 변환
        final_rate = first_rate / 10  # 10 으로 나눈 값을 실수형으로 가져옴.
        print(idx, ':', final_rate)
    print('=' * 50)
        

0 : 89.1
1 : 90.7
2 : 92.5
3 : 89.9
4 : 90.4
5 : 91.3
6 : 92.1
7 : 93.8
8 : 98.5
9 : 99.8
10 : 97.8
11 : 96.7
12 : 96.4
13 : 98.1
14 : 100.6
15 : 103.2
16 : 103.1
17 : 103.3
18 : 104.0
19 : 103.7
20 : 104.9
21 : 107.1
22 : 106.3
23 : 102.8
24 : 101.4
25 : 101.3
26 : 100.5
27 : 100.2
28 : 100.3
29 : 100.7
30 : 101.0
31 : 102.5
32 : 102.3
33 : 104.3
34 : 103.6
35 : 102.5
36 : 100.1
37 : 99.5
38 : 99.7
39 : 99.1
40 : 100.8
41 : 103.5
42 : 104.3
43 : 101.7
44 : 98.9
45 : 99.7
46 : 99.8
47 : 100.2
48 : 101.4
49 : 102.3
50 : 101.5
51 : 95.9
52 : 92.5
53 : 92.3
54 : 93.3
55 : 98.0
56 : 106.9
57 : 101.4
58 : 97.8
59 : 94.7
60 : 88.0
61 : 84.3
62 : 82.1
63 : 80.6
64 : 80.6
65 : 82.1
66 : 87.8
67 : 91.1
68 : 93.7
69 : 97.1
70 : 99.0
71 : 105.7
72 : 105.8
73 : 106.2
74 : 108.2
75 : 105.8
76 : 102.8
77 : 108.5
78 : 119.4
79 : 117.8
80 : 115.4
81 : 112.6
82 : 115.1
83 : 120.4
84 : 124.2
85 : 127.8
86 : 123.4
87 : 116.3
88 : 117.6
89 : 124.7
90 : 128.7
91 : 130.9
92 : 128.5
93 : 115.3
94 : 99.6
95 :

### 전체 코드 체크

In [8]:
area_panel = soup.find_all('div', class_='IBPageOne')[0]  # '분류', 'No' 칸 판넬
area_list = area_panel.find_all('tr')  # 'tr' 태그 찾기

# 수도권, 지방권 단어 찾기
for idx, each_area in enumerate(area_list[1:]):
    area_name = each_area.get_text()[1:]  # 이름

    # 매매 수급 지수 데이터 가져오기
    supply_demand_rate_panel = soup.find_all('div', class_='IBPageOne')[1]  # 1번째 'IBPageOne' 클래스 갸져오기
    rate_index = supply_demand_rate_panel.find_all('tr')[1:]  # 수도권 인덱스 = 0, 지방권 인덱스 = 1

    # 날짜 데이터 가져오기
    date_panel = soup.find('div', class_='IBHeadMid')  # 날짜 판텔
    body_panel = date_panel.find_all('tr')  # 'tr' 태그 모두 가져오기
    date_column = body_panel[1]  # 1번 째 'tr' 태그에 날짜가 있음.
    date_text = date_column.find_all('td')[1:129]  # 문자열로 가져오기

    # 각 월에 대한 매매 수급 지수 데이터 
    rate_list = rate_index[idx].find_all('td')[1:129]
    for month_idx, date in enumerate(date_text):
        year = int(date.get_text()[:4])  # 년도
        if len(date.get_text()) >= 9:
            month = int(date.get_text()[6:8])  # 월
        else:
            month = int(date.get_text()[6])  # 월

        # 매매 수급 지수 데이터 처리
        each_rate = rate_list[month_idx]  # 현재 월의 지수 
        first_rate = int(each_rate.get_text().replace('.', ''))  # '.' 없앤 후 정수로 변환
        final_rate = first_rate / 10  # 10 으로 나눈 값을 실수형으로 가져옴.

        print(f'{year}년 {month}월 {area_name}의 매매 수급 지수 = {final_rate}')
    print('=' * 50)
        
        

2014년 1월 수도권의 매매 수급 지수 = 89.1
2014년 2월 수도권의 매매 수급 지수 = 90.7
2014년 3월 수도권의 매매 수급 지수 = 92.5
2014년 4월 수도권의 매매 수급 지수 = 89.9
2014년 5월 수도권의 매매 수급 지수 = 90.4
2014년 6월 수도권의 매매 수급 지수 = 91.3
2014년 7월 수도권의 매매 수급 지수 = 92.1
2014년 8월 수도권의 매매 수급 지수 = 93.8
2014년 9월 수도권의 매매 수급 지수 = 98.5
2014년 10월 수도권의 매매 수급 지수 = 99.8
2014년 11월 수도권의 매매 수급 지수 = 97.8
2014년 12월 수도권의 매매 수급 지수 = 96.7
2015년 1월 수도권의 매매 수급 지수 = 96.4
2015년 2월 수도권의 매매 수급 지수 = 98.1
2015년 3월 수도권의 매매 수급 지수 = 100.6
2015년 4월 수도권의 매매 수급 지수 = 103.2
2015년 5월 수도권의 매매 수급 지수 = 103.1
2015년 6월 수도권의 매매 수급 지수 = 103.3
2015년 7월 수도권의 매매 수급 지수 = 104.0
2015년 8월 수도권의 매매 수급 지수 = 103.7
2015년 9월 수도권의 매매 수급 지수 = 104.9
2015년 10월 수도권의 매매 수급 지수 = 107.1
2015년 11월 수도권의 매매 수급 지수 = 106.3
2015년 12월 수도권의 매매 수급 지수 = 102.8
2016년 1월 수도권의 매매 수급 지수 = 101.4
2016년 2월 수도권의 매매 수급 지수 = 101.3
2016년 3월 수도권의 매매 수급 지수 = 100.5
2016년 4월 수도권의 매매 수급 지수 = 100.2
2016년 5월 수도권의 매매 수급 지수 = 100.3
2016년 6월 수도권의 매매 수급 지수 = 100.7
2016년 7월 수도권의 매매 수급 지수 = 101.0
2016년 8월 수도권의 매매 수급 지수 = 102.5
2016년 9월 수도권의 매매

## 테이블에 추가하기

In [11]:
sql_insert_query = """
INSERT INTO SUPPLY_TRENDS_BY_CATEGORY (YEAR, MONTH, CATEGORY, SUPPLY_INDEX)
VALUES (%s, %s, %s, %s)
"""

area_panel = soup.find_all('div', class_='IBPageOne')[0]  # '분류', 'No' 칸 판넬
area_list = area_panel.find_all('tr')  # 'tr' 태그 찾기

# 수도권, 지방권 단어 찾기
for idx, each_area in enumerate(area_list[1:]):
    area_name = each_area.get_text()[1:]  # 이름

    # 매매 수급 지수 데이터 가져오기
    supply_demand_rate_panel = soup.find_all('div', class_='IBPageOne')[1]  # 1번째 'IBPageOne' 클래스 갸져오기
    rate_index = supply_demand_rate_panel.find_all('tr')[1:]  # 수도권 인덱스 = 0, 지방권 인덱스 = 1

    # 날짜 데이터 가져오기
    date_panel = soup.find('div', class_='IBHeadMid')  # 날짜 판텔
    body_panel = date_panel.find_all('tr')  # 'tr' 태그 모두 가져오기
    date_column = body_panel[1]  # 1번 째 'tr' 태그에 날짜가 있음.
    date_text = date_column.find_all('td')[1:129]  # 문자열로 가져오기

    # 각 월에 대한 매매 수급 지수 데이터 
    rate_list = rate_index[idx].find_all('td')[1:129]
    for month_idx, date in enumerate(date_text):
        year = int(date.get_text()[:4])  # 년도
        if len(date.get_text()) >= 9:
            month = int(date.get_text()[6:8])  # 월
        else:
            month = int(date.get_text()[6])  # 월

        # 매매 수급 지수 데이터 처리
        each_rate = rate_list[month_idx]  # 현재 월의 지수 
        first_rate = int(each_rate.get_text().replace('.', ''))  # '.' 없앤 후 정수로 변환
        final_rate = first_rate / 10  # 10 으로 나눈 값을 실수형으로 가져옴.

        cur.execute(sql_insert_query, (year, month, area_name, final_rate))

remote.commit()
        
        

### 모두 종료

In [12]:
cur.close()
remote.close()
driver.close()